In [4]:
import pandas as pd
import geopandas as gpd
from pandas_geojson import to_geojson
from shapely.geometry import Point
import json


In [5]:
#Create path and red in CSV
path = './Resources/Wildfires_2.csv'
csv = pd.read_csv(path)
#Convert CSV into DF
df = pd.DataFrame(csv)

In [6]:
first_years_df = df[df["FIRE_YEAR"].between(2000, 2004)]
second_years_df = df[df["FIRE_YEAR"].between(2005, 2009)]
third_years_df = df[df["FIRE_YEAR"].between(2010, 2015)]

In [7]:
# Assuming you have loaded your DataFrame 'first_years_df'

# Store points for coordinates in a variable
geometry = [Point(xy) for xy in zip(first_years_df['LONGITUDE'], first_years_df['LATITUDE'])]

# Create GeoDataFrame and add geometry column
geo_first_years = gpd.GeoDataFrame(first_years_df, geometry=geometry)

# Drop unnecessary columns
gdf_first_years = geo_first_years.drop(['LATITUDE', 'LONGITUDE', 'index', 'STAT_CAUSE_DESCR', 'Fire_Date'], axis=1)
gdf_first_years = gdf_first_years.dropna()

# Sort Values
gdf_first_years.sort_values('FIRE_YEAR', inplace=True)

# Dropping unnecessary size classes 
gdf_2000 = gdf_first_years[~gdf_first_years['FIRE_SIZE_CLASS'].isin(['A', 'B', 'C'])]

# Convert non-spatial columns to properties
properties = gdf_2000.drop(['geometry'], axis=1).to_dict(orient='records')

# Convert Point geometries to GeoJSON-compatible format
gdf_2000['geometry'] = gdf_2000['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})

# Add properties to GeoJSON
geojson_data_first_years = {
    "features": []
}

for feature, prop in zip(gdf_2000['geometry'], properties):
    geojson_data_first_years['features'].append({
        "geometry": feature,
        "properties": prop
    })

# Save GeoJSON to a file
with open('first_years.geojson', 'w') as f:
    json.dump(geojson_data_first_years, f, indent=2)


C:\Users\ocean\AppData\Local\Temp\ipykernel_30428\4188593017.py:23: UserWarning: Geometry column does not contain geometry.
  gdf_2000['geometry'] = gdf_2000['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})
C:\Users\ocean\AppData\Roaming\Python\Python311\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
import geopandas as gpd
from shapely.geometry import Point
import json

# Assuming you have loaded your DataFrame 'second_years_df'

# Store points for coordinates in a variable
geometry = [Point(xy) for xy in zip(second_years_df['LONGITUDE'], second_years_df['LATITUDE'])]

# Create GeoDataFrame and add geometry column
geo_second_years = gpd.GeoDataFrame(second_years_df, geometry=geometry)

# Drop unnecessary columns
gdf_second_years = geo_second_years.drop(['LATITUDE', 'LONGITUDE', 'index', 'STAT_CAUSE_DESCR', 'Fire_Date'], axis=1)
gdf_second_years = gdf_second_years.dropna()

# Dropping unnecessary size classes 
gdf_2005 = gdf_second_years[~gdf_second_years['FIRE_SIZE_CLASS'].isin(['A', 'B', 'C'])]

# Sort Values
gdf_2005.sort_values('FIRE_YEAR', inplace=True)

# Convert non-spatial columns to properties
properties = gdf_2005.drop(['geometry'], axis=1).to_dict(orient='records')

# Convert Point geometries to GeoJSON-compatible format
gdf_2005['geometry'] = gdf_2005['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})

# Add properties to GeoJSON
geojson_data_second_years = {
    "features": []
}

for feature, prop in zip(gdf_2005['geometry'], properties):
    geojson_data_second_years['features'].append({
        "geometry": feature,
        "properties": prop
    })

# Save GeoJSON to a file
with open('second_years.geojson', 'w') as f:
    json.dump(geojson_data_second_years, f, indent=2)

C:\Users\ocean\AppData\Local\Temp\ipykernel_30428\1009050382.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_2005.sort_values('FIRE_YEAR', inplace=True)
C:\Users\ocean\AppData\Local\Temp\ipykernel_30428\1009050382.py:27: UserWarning: Geometry column does not contain geometry.
  gdf_2005['geometry'] = gdf_2005['geometry'].apply(lambda point: {'coordinates': (point.x, point.y)})
C:\Users\ocean\AppData\Roaming\Python\Python311\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [19]:
import geopandas as gpd
from shapely.geometry import Point
import json


def create_geojson(df, filename):
    # Store points for coordinates in a variable
    geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]

    # Create GeoDataFrame and add geometry column
    geo_df = gpd.GeoDataFrame(df, geometry=geometry)

    # Drop unnecessary columns
    gdf = geo_df.drop(['LATITUDE', 'LONGITUDE', 'index',
                      'STAT_CAUSE_DESCR', 'Fire_Date'], axis=1).copy()
    gdf = gdf.dropna()

    # Dropping unnecessary size classes
    gdf = gdf[~gdf['FIRE_SIZE_CLASS'].isin(['A', 'B', 'C'])]

    # Sort Values
    gdf.sort_values('FIRE_YEAR', inplace=True)

    # Convert non-spatial columns to properties
    properties = gdf.drop(['geometry'], axis=1).to_dict(orient='records')

    # Create a list for GeoJSON-compatible geometries
    geojson_geometries = gdf['geometry'].apply(lambda point: {'type': 'Point', 'coordinates': (
        point.x, point.y)} if point.is_valid else None).tolist()

    # Add properties to GeoJSON
    geojson_data = {
        "type": "FeatureCollection",
        "features": []
    }

    for feature, prop in zip(geojson_geometries, properties):
        # Check if the geometry is valid
        if feature is not None and 'coordinates' in feature and len(feature['coordinates']) == 2:
            geojson_data['features'].append({
                "type": "Feature",
                "geometry": feature,
                "properties": prop
            })

    # Save GeoJSON to a file
    with open(filename, 'w') as f:
        json.dump(geojson_data, f, indent=2)


# Call the function for each DataFrame
create_geojson(first_years_df, 'first_years.geojson')
create_geojson(second_years_df, 'second_years.geojson')
create_geojson(third_years_df, 'third_years.geojson')


In [14]:
,    

''